In [1]:
import os, glob, random, subprocess
proot = "."
program = proot + "/bc-1.07.1/bc/bc"
pseed = proot + "/bc-1.07.1/test"
pmutate = proot + "/mutate"

idxMinMutation = 0
idxMaxMutation = 4
numOfMutatedPrograms = 100

listOfSeeds = glob.glob (pseed + "/" + "*.b")
sizeOfSeeds = len(listOfSeeds)

def applyMutation(i, orgProgram):
    modProgram = ""
    if i == 0:
        modProgram = insert_random_character(orgProgram)
    elif i == 1:
        modProgram = delete_random_character(orgProgram)
    elif i == 2:
        modProgram = flip_random_character(orgProgram)
    elif i == 3:
        modProgram = walkingBitFlip(orgProgram)
    elif i == 4:
        modProgram = walkingByteFlip(orgProgram)
    return modProgram

In [2]:
# Two AFL Mutation [ref: AFL]
def walkingBitFlip(string):
    res = ""
    r = random.randrange(0, len(string))
    for i,c in enumerate(string):
        flip = ""
        # ASCII value 
        unicod = ord(c)
        #convert ASCII to binary
        binary =  bin(unicod)
        
        bitextend = binary
        if i == r: #bit-flip
            temp = binary[2:]
            lonG = 8 - len(temp)

            bitextend = "0b" + str(0)*lonG + temp

            SizeOfFlip = random.randrange(1, 5)
            for x in range(SizeOfFlip):
                if int(bitextend[x+2]):
                    flip += str(0)
                else: 
                    flip += str(1)
            bitextend = bitextend[:2] + flip + bitextend[2+SizeOfFlip:]
            
        #get character of binaries
        t =int(bitextend, 2)
        res += chr(t)
    return res

def walkingByteFlip(string):
    res = ""
    r = random.randrange(0, len(string))
    for i,c in enumerate(string):
        flip = ""
        # ASCII value 
        unicod = ord(c)
        #convert ASCII to binary
        binary =  bin(unicod)
        
        bitextend = binary
        if i == r: #byte flip 
            temp = binary[2:]
            lonG = 8 - len(temp)

            bitextend = "0b" + str(0)*lonG + temp

            SizeOfFlip = 8
            for x in range(SizeOfFlip):
                if int(bitextend[x+2]):
                    flip += str(0)
                else: 
                    flip += str(1)
            bitextend = bitextend[:2] + flip + bitextend[2+SizeOfFlip:]
            
        #get character of binaries
        t =int(bitextend, 2)
        res += chr(t)
        
    return res

In [3]:
# Three Random Mutation [ref: fuzzingbook]
def insert_random_character(s):
    pos = random.randint(0, len(s))
    random_character = chr(random.randrange(32, 127))
    return s[:pos] + random_character + s[pos:]

def delete_random_character(s):
    pos = random.randint(0, len(s) - 1)
    return s[:pos] + s[pos + 1:]

def flip_random_character(s):
    pos = random.randint(0, len(s) - 1)
    old_c = s[pos]
    bit = 1 << random.randint(0, 6)
    new_c = chr(ord(old_c) ^ bit)
    return s[:pos] + new_c + s[pos + 1:]

In [4]:
# Generating mutated programs
for i in range(idxMinMutation, idxMaxMutation + 1):
    for j in range(0, numOfMutatedPrograms):
        idxOfSeed = random.randrange(0, sizeOfSeeds)
        programStr = ""
        with open(listOfSeeds[idxOfSeed], 'r') as fSeed:
            orgProgram = fSeed.read()
            modProgram = applyMutation(i, orgProgram)
            mfilename = pmutate + "/" + "m"+str(i+1) + "/" + str(j+1)+".b"
            if not os.path.exists(os.path.dirname(mfilename)):
                os.makedirs(os.path.dirname(mfilename))
            with open(mfilename, "w") as fMutate:
                fMutate.write(str(modProgram) + "\n")

In [5]:
# Evaluating mutated programs
for i in range(idxMinMutation, idxMaxMutation + 1):
    count = 0
    for j in range(0, numOfMutatedPrograms):
        mfilename = pmutate + "/" + "m"+str(i+1) + "/" + str(j+1)+".b"
        try:
            result = subprocess.run([program, mfilename],
                                    stdin=subprocess.DEVNULL,
                                    stdout=subprocess.PIPE,
                                    stderr=subprocess.PIPE,
                                    timeout=3,
                                    universal_newlines=True)
        except:
            pass
        if len(result.stderr) > 0: count = count + 1
    print("Mutation " + str(i+1) + ", # invalid inputs = " + str(count) )

Mutation 1, # invalid inputs = 78
Mutation 2, # invalid inputs = 71
Mutation 3, # invalid inputs = 81
Mutation 4, # invalid inputs = 95
Mutation 5, # invalid inputs = 97


In [6]:
# Generating multiple mutated programs with Random Only
for j in range(0, numOfMutatedPrograms):
    idxOfSeed = random.randrange(0, sizeOfSeeds)
    programStr = ""
    with open(listOfSeeds[idxOfSeed], 'r') as fSeed:
        orgProgram = fSeed.read()
        modProgram = applyMutation(0, orgProgram)
        modProgram = applyMutation(1, modProgram)
        modProgram = applyMutation(2, modProgram)
        mfilename = pmutate + "/" + "m"+str(123) + "/" + str(j+1)+".b"
        if not os.path.exists(os.path.dirname(mfilename)):
            os.makedirs(os.path.dirname(mfilename))
        with open(mfilename, "w") as fMutate:
            fMutate.write(modProgram + "\n")

In [7]:
# Generating multiple mutated programs with AFL Only
for j in range(0, numOfMutatedPrograms):
    idxOfSeed = random.randrange(0, sizeOfSeeds)
    programStr = ""
    with open(listOfSeeds[idxOfSeed], 'r') as fSeed:
        orgProgram = fSeed.read()
        modProgram = applyMutation(3, orgProgram)
        modProgram = applyMutation(4, modProgram)
        mfilename = pmutate + "/" + "m"+str(45) + "/" + str(j+1)+".b"
        if not os.path.exists(os.path.dirname(mfilename)):
            os.makedirs(os.path.dirname(mfilename))
        with open(mfilename, "w") as fMutate:
            fMutate.write(modProgram + "\n")

In [8]:
# Generating multiple mutated programs with Random + AFL
for j in range(0, numOfMutatedPrograms):
    idxOfSeed = random.randrange(0, sizeOfSeeds)
    programStr = ""
    with open(listOfSeeds[idxOfSeed], 'r') as fSeed:
        orgProgram = fSeed.read()
        modProgram = applyMutation(0, orgProgram)
        for i in range(1,5):
            modProgram = applyMutation(i, modProgram)
        mfilename = pmutate + "/" + "m"+str(12345) + "/" + str(j+1)+".b"
        if not os.path.exists(os.path.dirname(mfilename)):
            os.makedirs(os.path.dirname(mfilename))
        with open(mfilename, "w") as fMutate:
            fMutate.write(modProgram + "\n")

In [9]:
# Evaluating multiple mutated programs 
mulmut = ["123","45","12345"]
for i in mulmut:
    count = 0
    for j in range(0, numOfMutatedPrograms):
        mfilename = pmutate + "/" + "m"+str(i) + "/" + str(j+1)+".b"
        try:
            result = subprocess.run([program, mfilename],
                                    stdin=subprocess.DEVNULL,
                                    stdout=subprocess.PIPE,
                                    stderr=subprocess.PIPE,
                                    timeout=3,
                                    universal_newlines=True)
        except:
            pass
        if len(result.stderr) > 0: count = count + 1
    print("Mutation " + str(i) + ", # invalid inputs = " + str(count) )

Mutation 123, # invalid inputs = 95
Mutation 45, # invalid inputs = 98
Mutation 12345, # invalid inputs = 100


In [11]:
# Generating mutated programs on operator/number
for j in range(0, numOfMutatedPrograms):
    idxOfSeed = random.randrange(0, sizeOfSeeds)
    programStr = ""
    with open(listOfSeeds[idxOfSeed], 'r') as fSeed:
        orgProgram = fSeed.read()
        modProgOp, modProgNum = "", ""
        opList = ['+', '-', '*', '/']
        dgList = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
        for c in orgProgram:
            
            #operators
            if len(opList)>0 and (c in opList):
                opList.remove(c)
                nc = ""
                if len(opList)>0:
                    nc = str(random.choice(opList))
                modProgOp = modProgOp + str(nc)
            else:
                modProgOp = modProgOp + str(c)
                
            #numbers
            if len(dgList)>0 and (c in dgList):
                dgList.remove(c)
                nc = ""
                if len(dgList)>0:
                    nc = str(random.choice(dgList))
                modProgNum = modProgNum + str(nc)
            else:
                modProgNum = modProgNum + str(c)
        
        #operators
        mfileOp = pmutate + "/" + "mOp" + "/" + str(j+1)+".b"
        if not os.path.exists(os.path.dirname(mfileOp)):
            os.makedirs(os.path.dirname(mfileOp))
        with open(mfileOp, "w") as fMutate:
            fMutate.write(modProgOp + "\n")
        
        #numbers
        mfileNum = pmutate + "/" + "mNum" + "/" + str(j+1)+".b"
        if not os.path.exists(os.path.dirname(mfileNum)):
            os.makedirs(os.path.dirname(mfileNum))
        with open(mfileNum, "w") as fMutate:
            fMutate.write(modProgNum + "\n")
            


In [12]:
# Evaluating operator/number mutated programs 
mulmut = ["Op","Num"]
for i in mulmut:
    count = 0
    for j in range(0, numOfMutatedPrograms):
        mfilename = pmutate + "/" + "m"+str(i) + "/" + str(j+1)+".b"
        try:
            result = subprocess.run([program, mfilename],
                                    stdin=subprocess.DEVNULL,
                                    stdout=subprocess.PIPE,
                                    stderr=subprocess.PIPE,
                                    timeout=3,
                                    universal_newlines=True)
        except:
            pass
        if len(result.stderr) > 0: count = count + 1
    print("Mutation " + str(i) + ", # invalid inputs = " + str(count) )

Mutation Op, # invalid inputs = 100
Mutation Num, # invalid inputs = 45
